#### This notebook demonstrates the use of the Reject Option Classification (ROC) post-processing algorithm for bias mitigation.
- The debiasing function used is implemented in the `RejectOptionClassification` class.
- Divide the dataset into training, validation, and testing partitions.
- Train classifier on original training data.
- Estimate the optimal classification threshold, that maximizes balanced accuracy without fairness constraints.
- Estimate the optimal classification threshold, and the critical region boundary (ROC margin) using a validation set for the desired constraint on fairness. The best parameters are those that maximize the classification threshold while satisfying the fairness constraints.
- The constraints can be used on the following fairness measures:
    * Statistical parity difference on the predictions of the classifier
    * Average odds difference for the classifier
    * Equal opportunity difference for the classifier
- Determine the prediction scores for testing data. Using the estimated optimal classification threshold, compute accuracy and fairness metrics.
- Using the determined optimal classification threshold and the ROC margin, adjust the predictions. Report accuracy and fairness metric on the new predictions.

In [7]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
from tqdm import tqdm
from warnings import warn

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
import pandas as pd

#### Load dataset and specify options


Huangrui's ADULT PREPROCESS

Only modify the sex and race to 1/0

With favorable one as 1. 

keep all other features

Race Is not dummy columned, thus we have fewer columns


In [8]:
def load_preproc_data_adult(protected_attributes=None):
    def custom_preprocessing(df):
            """The custom pre-processing function is adapted from
                https://github.com/fair-preprocessing/nips2017/blob/master/Adult/code/Generate_Adult_Data.ipynb
                If sub_samp != False, then return smaller version of dataset truncated to tiny_test data points.
            """

            def group_race(x):
                if x == "White":
                    return 1.0
                else:
                    return 0.0
            # Rename income variable
            df['income-per-year'] = df['income-per-year'].replace(to_replace='>50K.', value='>50K', regex=True)
            df['income-per-year'] = df['income-per-year'].replace(to_replace='<=50K.', value='<=50K', regex=True)
            df['sex'] = df['sex'].replace({'Female': 0.0, 'Male': 1.0})
            df['race'] = df['race'].apply(lambda x: group_race(x))
            return df


    D_features = protected_attributes
    all_privileged_classes = {"sex": [1.0],
                                "race": [1.0]}

    # protected attribute maps
    all_protected_attribute_maps = {"sex": {1.0: 'Male', 0.0: 'Female'},
                                    "race": {1.0: 'White', 0.0: 'Non-white'}}

    return AdultDataset(
            favorable_classes=['>50K', '>50K.'],
            protected_attribute_names=D_features,
            privileged_classes=[all_privileged_classes[x] for x in D_features],
            instance_weights_name=None,
            features_to_drop=[],
            na_values=[],
            metadata={'label_maps': [{1.0: '>50K', 0.0: '<=50K'}],
                    'protected_attribute_maps': [all_protected_attribute_maps[x]
                                    for x in D_features]},
            custom_preprocessing=custom_preprocessing)

In [9]:
## import dataset
dataset_used = "adult" # "adult", "german", "compas"
protected_attribute_used = 1 # 1, 2

if dataset_used == "adult":
#     dataset_orig = AdultDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
        dataset_orig = load_preproc_data_adult(['sex'])
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]
        dataset_orig = load_preproc_data_adult(['race'])
        
# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
#random seed for calibrated equal odds prediction
random_seed = 12345679
np.random.seed(random_seed)

# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

In [10]:
# 5 fold cross validation
Z =  dataset_orig.split(5, shuffle=True,seed = random_seed)
# i th fold
dataset_train1 = Z[0].copy()
dataset_train1.features = np.concatenate((Z[0].features,Z[1].features,Z[2].features,Z[3].features),axis=0)
dataset_train1.scores = np.concatenate((Z[0].scores,Z[1].scores,Z[2].scores,Z[3].scores),axis=0)
dataset_train1.labels = np.concatenate((Z[0].labels,Z[1].labels,Z[2].labels,Z[3].labels),axis=0)
dataset_train1.protected_attributes = np.concatenate((Z[0].protected_attributes,Z[1].protected_attributes,Z[2].protected_attributes,Z[3].protected_attributes),axis=0)
dataset_train1.instance_weights = np.concatenate((Z[0].instance_weights,Z[1].instance_weights,Z[2].instance_weights,Z[3].instance_weights),axis=0)
dataset_train1.instance_names = np.concatenate((Z[0].instance_names,Z[1].instance_names,Z[2].instance_names,Z[3].instance_names),axis=0)
dataset_train1.metadata = Z[0].metadata.copy()
dataset_test1= Z[4].copy()

dataset_train2 = Z[1].copy()
dataset_train2.features = np.concatenate((Z[1].features,Z[2].features,Z[3].features,Z[4].features),axis=0)
dataset_train2.scores = np.concatenate((Z[1].scores,Z[2].scores,Z[3].scores,Z[4].scores),axis=0)
dataset_train2.labels = np.concatenate((Z[1].labels,Z[2].labels,Z[3].labels,Z[4].labels),axis=0)
dataset_train2.protected_attributes = np.concatenate((Z[1].protected_attributes,Z[2].protected_attributes,Z[3].protected_attributes,Z[4].protected_attributes),axis=0)
dataset_train2.instance_weights = np.concatenate((Z[1].instance_weights,Z[2].instance_weights,Z[3].instance_weights,Z[4].instance_weights),axis=0)
dataset_train2.instance_names = np.concatenate((Z[1].instance_names,Z[2].instance_names,Z[3].instance_names,Z[4].instance_names),axis=0)
dataset_train2.metadata = Z[1].metadata.copy()
dataset_test2= Z[0].copy()

dataset_train3 = Z[2].copy()
dataset_train3.features = np.concatenate((Z[2].features,Z[3].features,Z[4].features,Z[0].features),axis=0)
dataset_train3.scores = np.concatenate((Z[2].scores,Z[3].scores,Z[4].scores,Z[0].scores),axis=0)
dataset_train3.labels = np.concatenate((Z[2].labels,Z[3].labels,Z[4].labels,Z[0].labels),axis=0)
dataset_train3.protected_attributes = np.concatenate((Z[2].protected_attributes,Z[3].protected_attributes,Z[4].protected_attributes,Z[0].protected_attributes),axis=0)
dataset_train3.instance_weights = np.concatenate((Z[2].instance_weights,Z[3].instance_weights,Z[4].instance_weights,Z[0].instance_weights),axis=0)
dataset_train3.instance_names = np.concatenate((Z[2].instance_names,Z[3].instance_names,Z[4].instance_names,Z[0].instance_names),axis=0)
dataset_train3.metadata = Z[2].metadata.copy()
dataset_test3= Z[1].copy()

dataset_train4 = Z[3].copy()
dataset_train4.features = np.concatenate((Z[3].features,Z[4].features,Z[0].features,Z[1].features),axis=0)
dataset_train4.scores = np.concatenate((Z[3].scores,Z[4].scores,Z[0].scores,Z[1].scores),axis=0)
dataset_train4.labels = np.concatenate((Z[3].labels,Z[4].labels,Z[0].labels,Z[1].labels),axis=0)
dataset_train4.protected_attributes = np.concatenate((Z[3].protected_attributes,Z[4].protected_attributes,Z[0].protected_attributes,Z[1].protected_attributes),axis=0)
dataset_train4.instance_weights = np.concatenate((Z[3].instance_weights,Z[4].instance_weights,Z[0].instance_weights,Z[1].instance_weights),axis=0)
dataset_train4.instance_names = np.concatenate((Z[3].instance_names,Z[4].instance_names,Z[0].instance_names,Z[1].instance_names),axis=0)
dataset_train4.metadata = Z[3].metadata.copy()
dataset_test4= Z[2].copy()

dataset_train5 = Z[4].copy()
dataset_train5.features = np.concatenate((Z[4].features,Z[0].features,Z[1].features,Z[2].features),axis=0)
dataset_train5.scores = np.concatenate((Z[4].scores,Z[0].scores,Z[1].scores,Z[2].scores),axis=0)
dataset_train5.labels = np.concatenate((Z[4].labels,Z[0].labels,Z[1].labels,Z[2].labels),axis=0)
dataset_train5.protected_attributes = np.concatenate((Z[4].protected_attributes,Z[0].protected_attributes,Z[1].protected_attributes,Z[2].protected_attributes),axis=0)
dataset_train5.instance_weights = np.concatenate((Z[4].instance_weights,Z[0].instance_weights,Z[1].instance_weights,Z[2].instance_weights),axis=0)
dataset_train5.instance_names = np.concatenate((Z[4].instance_names,Z[0].instance_names,Z[1].instance_names,Z[2].instance_names),axis=0)
dataset_train5.metadata = Z[4].metadata.copy()
dataset_test5= Z[3].copy()

## save the split dataset


In [11]:
# protected_name = "sex"
# dataset_train1.convert_to_dataframe()[0].to_csv("Huangrui/adult/{}/adult_train1.csv".format(protected_name),index=False)
# dataset_test1.convert_to_dataframe()[0].to_csv("Huangrui/adult/{}/adult_test1.csv".format(protected_name),index=False)
# dataset_train2.convert_to_dataframe()[0].to_csv("Huangrui/adult/{}/adult_train2.csv".format(protected_name),index=False)
# dataset_test2.convert_to_dataframe()[0].to_csv("Huangrui/adult/{}/adult_test2.csv".format(protected_name),index=False)
# dataset_train3.convert_to_dataframe()[0].to_csv("Huangrui/adult/{}/adult_train3.csv".format(protected_name),index=False)
# dataset_test3.convert_to_dataframe()[0].to_csv("Huangrui/adult/{}/adult_test3.csv".format(protected_name),index=False)
# dataset_train4.convert_to_dataframe()[0].to_csv("Huangrui/adult/{}/adult_train4.csv".format(protected_name),index=False)
# dataset_test4.convert_to_dataframe()[0].to_csv("Huangrui/adult/{}/adult_test4.csv".format(protected_name),index=False)
# dataset_train5.convert_to_dataframe()[0].to_csv("Huangrui/adult/{}/adult_train5.csv".format(protected_name),index=False)
# dataset_test5.convert_to_dataframe()[0].to_csv("Huangrui/adult/{}/adult_test5.csv".format(protected_name),index=False)


#### Split into train, test and validation

change the dataset_train[i] for specific K

In [12]:
dataset_orig_train_valid= dataset_train1.copy()

In [13]:
dataset_orig_test= dataset_test1.copy()

In [14]:
# Get the dataset and split into train and test
dataset_orig_train, dataset_orig_valid = dataset_orig_train_valid.split([0.8], shuffle=True)

#### Clean up training data and display properties of the data

In [15]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(31259, 103)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['sex']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['age', 'fnlwgt', 'education-num', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'workclass=?', 'workclass=Federal-gov', 'workclass=Local-gov', 'workclass=Never-worked', 'workclass=Private', 'workclass=Self-emp-inc', 'workclass=Self-emp-not-inc', 'workclass=State-gov', 'workclass=Without-pay', 'education=10th', 'education=11th', 'education=12th', 'education=1st-4th', 'education=5th-6th', 'education=7th-8th', 'education=9th', 'education=Assoc-acdm', 'education=Assoc-voc', 'education=Bachelors', 'education=Doctorate', 'education=HS-grad', 'education=Masters', 'education=Preschool', 'education=Prof-school', 'education=Some-college', 'marital-status=Divorced', 'marital-status=Married-AF-spouse', 'marital-status=Married-civ-spouse', 'marital-status=Married-spouse-absent', 'marital-status=Never-married', 'marital-status=Separated', 'marital-status=Widowed', 'occupation=?', 'occupation=Adm-clerical', 'occupation=Armed-Forces', 'occupation=Craft-repair', 'occupation=Exec-man

#### Metric for original training data

In [16]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.192845


### Train classifier on original data

In [17]:
dataset_orig_train

               instance weights features                               \
                                                                        
                                     age    fnlwgt education-num race   
instance names                                                          
12295                       1.0     23.0  132556.0           9.0  1.0   
4676                        1.0     56.0  146391.0           9.0  1.0   
6270                        1.0     56.0  179127.0           6.0  1.0   
9398                        1.0     26.0   61996.0           9.0  1.0   
23206                       1.0     45.0  327886.0          15.0  1.0   
...                         ...      ...       ...           ...  ...   
29929                       1.0     40.0  114580.0           9.0  1.0   
36357                       1.0     43.0  163455.0           9.0  1.0   
34348                       1.0     27.0  150080.0          13.0  1.0   
6612                        1.0     40.0  352971.0 

In [18]:
# Logistic regression classifier and predictions
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()

lmod = LogisticRegression()
lmod.fit(X_train, y_train)
y_train_pred = lmod.predict(X_train)

# positive class index
pos_ind = np.where(lmod.classes_ == dataset_orig_train.favorable_label)[0][0]
print(pos_ind)
dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_train_pred.labels = y_train_pred

1


In [30]:
y_train_pred

array([0., 0., 0., ..., 0., 1., 0.])

#### Obtain scores for validation and test sets

In [19]:
dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
X_valid = scale_orig.transform(dataset_orig_valid_pred.features)
y_valid = dataset_orig_valid_pred.labels
dataset_orig_valid_pred.scores = lmod.predict_proba(X_valid)[:,pos_ind].reshape(-1,1)

dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
X_test = scale_orig.transform(dataset_orig_test_pred.features)
y_test = dataset_orig_test_pred.labels
dataset_orig_test_pred.scores = lmod.predict_proba(X_test)[:,pos_ind].reshape(-1,1)

In [29]:
lmod.predict_proba(X_valid)

array([[0.82873812, 0.17126188],
       [0.07850951, 0.92149049],
       [0.98867822, 0.01132178],
       ...,
       [0.94423378, 0.05576622],
       [0.99061177, 0.00938823],
       [0.97762072, 0.02237928]])

### Find the optimal parameters from the validation set

#### Best threshold for classification only (no fairness)

In [20]:
num_thresh = 100
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):
    
    fav_inds = dataset_orig_valid_pred.scores > class_thresh
    dataset_orig_valid_pred.labels[fav_inds] = dataset_orig_valid_pred.favorable_label
    dataset_orig_valid_pred.labels[~fav_inds] = dataset_orig_valid_pred.unfavorable_label
    
    classified_metric_orig_valid = ClassificationMetric(dataset_orig_valid,
                                             dataset_orig_valid_pred, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
    
    ba_arr[idx] = 0.5*(classified_metric_orig_valid.true_positive_rate()\
                       +classified_metric_orig_valid.true_negative_rate())

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)

Best balanced accuracy (no fairness constraints) = 0.8269
Optimal classification threshold (no fairness constraints) = 0.2476


#### Estimate optimal parameters for the ROC method

In [21]:
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name=metric_name,
                                  metric_ub=metric_ub, metric_lb=metric_lb)
ROC = ROC.fit(dataset_orig_valid, dataset_orig_valid_pred)

In [22]:
print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

Optimal classification threshold (with fairness constraints) = 0.1783
Optimal ROC margin = 0.0400


### Predictions from Validation Set

In [23]:
# Metrics for the test set
fav_inds = dataset_orig_valid_pred.scores > best_class_thresh
dataset_orig_valid_pred.labels[fav_inds] = dataset_orig_valid_pred.favorable_label
dataset_orig_valid_pred.labels[~fav_inds] = dataset_orig_valid_pred.unfavorable_label

display(Markdown("#### Validation set"))
display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

metric_valid_bef = compute_metrics(dataset_orig_valid, dataset_orig_valid_pred, 
                unprivileged_groups, privileged_groups)

#### Validation set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Balanced accuracy = 0.8269
Statistical parity difference = -0.3222
Disparate impact = 0.2820
Average odds difference = -0.1682
Equal opportunity difference = -0.1225
Theil index = 0.0795


In [24]:
# Transform the validation set
dataset_transf_valid_pred = ROC.predict(dataset_orig_valid_pred)

display(Markdown("#### Validation set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_valid_aft = compute_metrics(dataset_orig_valid, dataset_transf_valid_pred, 
                unprivileged_groups, privileged_groups)

#### Validation set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 0.8258
Statistical parity difference = -0.3024
Disparate impact = 0.3720
Average odds difference = -0.1219
Equal opportunity difference = -0.0400
Theil index = 0.0753


In [25]:
# Testing: Check if the metric optimized has not become worse
# assert np.abs(metric_valid_aft[metric_name]) <= np.abs(metric_valid_bef[metric_name])

### Predictions from Test Set

In [26]:
# Metrics for the test set
fav_inds = dataset_orig_test_pred.scores > best_class_thresh
dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label

display(Markdown("#### Test set"))
display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                unprivileged_groups, privileged_groups)

#### Test set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Balanced accuracy = 0.8170
Statistical parity difference = -0.3340
Disparate impact = 0.2732
Average odds difference = -0.2108
Equal opportunity difference = -0.1970
Theil index = 0.0833


In [27]:
# Metrics for the transformed test set
dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)

display(Markdown("#### Test set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                unprivileged_groups, privileged_groups)

#### Test set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 0.8164
Statistical parity difference = -0.3075
Disparate impact = 0.3723
Average odds difference = -0.1625
Equal opportunity difference = -0.1195
Theil index = 0.0784


In [28]:
metric_test_aft["Balanced accuracy"]
metric_test_aft["Equal opportunity difference"]
print("The Error for the test dataset is {:.4}".format(1-metric_test_aft["Balanced accuracy"]))
print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test_aft["Equal opportunity difference"]))

The Error for the test dataset is 0.1836
The Equal opportunity difference for the test dataset is -0.1195
